In [23]:
import numpy as np
from tensorflow.keras.models import load_model

from tensorflow.keras import layers,models
import librosa

In [37]:
X_train_l = np.load('X_train_gender.npy')
X_test_l = np.load('X_test_gender.npy')
y_train_l = np.load('y_train_gender.npy')
y_test_l = np.load('y_test_gender.npy')

In [38]:
model_gender_4 = models.Sequential()

model_gender_4.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 130, 1)))
model_gender_4.add(layers.BatchNormalization())
model_gender_4.add(layers.MaxPooling2D((2, 2)))

model_gender_4.add(layers.Conv2D(128, (3, 3), activation='relu'))
model_gender_4.add(layers.BatchNormalization())
model_gender_4.add(layers.MaxPooling2D((2, 2)))

model_gender_4.add(layers.Conv2D(256, (3, 3), activation='relu'))
model_gender_4.add(layers.BatchNormalization())
model_gender_4.add(layers.MaxPooling2D((2, 2)))

model_gender_4.add(layers.Flatten())

model_gender_4.add(layers.Dense(128, activation='relu'))
model_gender_4.add(layers.BatchNormalization())
model_gender_4.add(layers.Dense(64, activation='relu'))
model_gender_4.add(layers.Dense(2, activation='softmax'))

model_gender_4.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])


In [39]:
ep=3
history_language = model_gender_4.fit(X_train_l, y_train_l, epochs=ep, validation_data=(X_test_l, y_test_l), batch_size=32)

Epoch 1/3
335/335 [==============================] - 198s 591ms/step - loss: 0.3202 - accuracy: 0.8261 - val_loss: 18.2154 - val_accuracy: 0.3529
Epoch 2/3
335/335 [==============================] - 417s 1s/step - loss: 0.3385 - accuracy: 0.8229 - val_loss: 71.1792 - val_accuracy: 0.3521
Epoch 3/3
335/335 [==============================] - 208s 622ms/step - loss: 0.3737 - accuracy: 0.7942 - val_loss: 3.0369 - val_accuracy: 0.3525


In [42]:
model_gender_4.save("models/genderModel3.1.h5")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [41]:
import json

# Save history to a JSON file
with open('history_gender_3.1.json', 'w') as json_file:
    json.dump(history_language.history, json_file)

In [55]:
model_gender_4 = load_model("models/genderModel3.1.h5") 

In [56]:
audio_file_path = '/Users/dheemankumar/github/audio-ai/female_eng.wav' # Adjust the path as needed
audio, sample_rate = librosa.load(audio_file_path, sr=22050)  # Load audio with a sample rate of 22050

mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sample_rate)

# Convert to decibels (log scale)
mel_spectrogram_db = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)

mel_spectrogram_db_with_channel = np.expand_dims(mel_spectrogram_db, axis=-1)

input_data = mel_spectrogram_db_with_channel.reshape(1, 128, 130, 1)
# Append the processed audio data and label to the lists
print(input_data.shape)

(1, 128, 130, 1)


In [57]:
ans=model_gender_4.predict(input_data)

1/1 [==============================] - 0s 80ms/step


In [58]:
male=ans[0][0]
female=ans[0][1]

In [59]:
if (male>(1.5*female)):
    print("Gender : Male")
elif (female>(1.5*male)):
    print("Gender : Female")
else:
    print("not conform")
    

Gender : Female
